# Library

In [ ]:
import requests
from bs4 import BeautifulSoup as bs

import time, os, re
from datetime import datetime
import pandas as pd

# 페이지 접속

In [4]:
url = 'https://finance.naver.com/' # 네이버 증권 > 홈

## requests 테스트
- 페이지 접속 가능 여부확인
    - 가능할 경우 출력 : <Response [200]>

In [114]:
req = requests.get(url)
print(req)

# html = req.content.decode('utf-8') # 한글 깨짐 해결 코드
html = req.text
soup = bs(html, 'html.parser')

# 현재 시간 가져오기
current_time = datetime.now()
# current_time = datetime.now().strftime('%Y%m%d_%H%M%S')
print(current_time)
print(soup.title.text)

<Response [200]>
2025-01-03 13:30:53.702250
네이버페이 증권


# 정보 수집

## 탭1(거래상위)

In [95]:
# TOP 종목
section_sise_top = soup.select_one('div#content div.section_sise_top')
section_sise_top

<div class="section_sise_top">
<h2 class="h_type"><span>TOP 종목</span></h2>
<ul class="tab_area sise_top1">
<li class="tab1"><a class="is_active" href="#" onclick="clickcr(this, 'spe.sale', '', '', event);return false;"><span>거래상위</span></a></li>
<li class="tab2"><a href="#" onclick="clickcr(this, 'spe.ceiling', '', '', event);return false;"><span>상승</span></a></li>
<li class="tab3"><a href="#" onclick="clickcr(this, 'spe.bottom', '', '', event);return false;"><span>하락</span></a></li>
<li class="tab4"><a href="#" onclick="clickcr(this, 'spe.value', '', '', event);return false;"><span>시가총액 상위</span></a></li>
</ul>
<div class="group_type is_active">
<table class="tbl_home">
<caption>거래상위 TOP 종목<span>종목명에 대한 현재가, 전일대비, 등락률로 구분되어 있습니다.</span></caption>
<colgroup>
<col width="40%"/>
<col/>
</colgroup>
<thead>
<tr>
<th scope="col">종목명</th>
<th scope="col">현재가</th>
<th scope="col">전일대비</th>
<th scope="col">등락률</th>
</tr>
</thead>
<tbody id="_topItems1">
<tr class="down">
<th scope="row"><a hre

In [96]:
group_types = section_sise_top.select('div.group_type')
len(group_types)

4

In [97]:
group_type = group_types[0]
group_type

<div class="group_type is_active">
<table class="tbl_home">
<caption>거래상위 TOP 종목<span>종목명에 대한 현재가, 전일대비, 등락률로 구분되어 있습니다.</span></caption>
<colgroup>
<col width="40%"/>
<col/>
</colgroup>
<thead>
<tr>
<th scope="col">종목명</th>
<th scope="col">현재가</th>
<th scope="col">전일대비</th>
<th scope="col">등락률</th>
</tr>
</thead>
<tbody id="_topItems1">
<tr class="down">
<th scope="row"><a href="/item/main.naver?code=252670" onclick="clickcr(this, 'spe.slist', '252670', '1', event);">KODEX 200선물인버스2X</a></th>
<td>2,460</td>
<td><em class="bu_p bu_pdn"><span class="blind">하락</span></em> 135</td>
<td>
<em class="down">
				-5.20%
				</em>
</td>
</tr>
<tr class="down">
<th scope="row"><a href="/item/main.naver?code=530036" onclick="clickcr(this, 'spe.slist', '530036', '2', event);">삼성 인버스 2X WTI원유 선물 ETN</a></th>
<td>88</td>
<td><em class="bu_p bu_pdn"><span class="blind">하락</span></em> 5</td>
<td>
<em class="down">
				-5.38%
				</em>
</td>
</tr>
<tr class="up">
<th scope="row"><a href="/item/main.nav

In [98]:
# stock_lines = group_type.select('tbody#_topItems1 tr')
stock_lines = group_type.select('tbody tr')
len(stock_lines)

15

In [99]:
# 1번째 종목
one_stock = stock_lines[0]
one_stock

<tr class="down">
<th scope="row"><a href="/item/main.naver?code=252670" onclick="clickcr(this, 'spe.slist', '252670', '1', event);">KODEX 200선물인버스2X</a></th>
<td>2,460</td>
<td><em class="bu_p bu_pdn"><span class="blind">하락</span></em> 135</td>
<td>
<em class="down">
				-5.20%
				</em>
</td>
</tr>

In [100]:
# 종목명
stock_name = one_stock.select_one('th a').text
stock_name

'KODEX 200선물인버스2X'

In [101]:
# 현재가
current_price = int(one_stock.select_one('td').text.replace(',', ''))
current_price

2460

# 🔧 전일 대비 상승, 하락 출력
- 빨간색 숫자는 그대로
- 파란색 숫자는 음수로

In [102]:
# 전일대비
raw_compare_yesterday = one_stock.select('td')[1].text
compare_yesterday = int(re.sub('[^0-9]', '', raw_compare_yesterday))
compare_yesterday

135

In [103]:
# 등락률
raw_updown_ratio = one_stock.select('td')[2].text.strip()
updown_ratio = float(re.sub('[^0-9.-]', '', raw_updown_ratio))
updown_ratio

-5.2

### 데이터 수집

In [104]:
stock_name_list = []
current_price_list = []
compare_yesterday_list = []
updown_ratio_list = []
for one_stock in stock_lines:
    # 종목명
    stock_name = one_stock.select_one('th a').text
    
    # 현재가
    current_price = int(one_stock.select_one('td').text.replace(',', ''))

    # 전일대비
    raw_compare_yesterday = one_stock.select('td')[1].text
    compare_yesterday = int(re.sub('[^0-9]', '', raw_compare_yesterday))

    # 등락률
    raw_updown_ratio = one_stock.select('td')[2].text.strip()
    updown_ratio = float(re.sub('[^0-9.-]', '', raw_updown_ratio))
    
    # 리스트에 추가
    stock_name_list.append(stock_name)
    current_price_list.append(current_price)
    compare_yesterday_list.append(compare_yesterday)
    updown_ratio_list.append(updown_ratio)

In [105]:
len(stock_name_list), len(current_price_list), len(compare_yesterday_list), len(updown_ratio_list)

(15, 15, 15, 15)

In [106]:
tab1_df = pd.DataFrame(
    {
        '수집시간': current_time,
        '종목명': stock_name_list,
        '현재가': current_price_list,
        '전일대비': compare_yesterday_list,
        '등락률': updown_ratio_list
    }
)
tab1_df

,수집시간,종목명,현재가,전일대비,등락률
0,2025-01-03 13:24:45.868512,KODEX 200선물인버스2X,2460,135,-5.20
1,2025-01-03 13:24:45.868512,삼성 인버스 2X WTI원유 선물 ETN,88,5,-5.38
2,2025-01-03 13:24:45.868512,오리엔트바이오,1861,125,7.20
3,2025-01-03 13:24:45.868512,KODEX 코스닥150레버리지,7225,505,7.51
4,2025-01-03 13:24:45.868512,골든센츄리,4,1,-20.00
5,2025-01-03 13:24:45.868512,형지I&C,1153,134,13.15
6,2025-01-03 13:24:45.868512,KODEX 코스닥150선물인버스,3990,155,-3.74
7,2025-01-03 13:24:45.868512,아톤,7990,1840,29.92
8,2025-01-03 13:24:45.868512,동원금속,2560,85,3.43
9,2025-01-03 13:24:45.868512,신한 인버스 2X WTI원유 선물 ETN(H),72,3,-4.00


In [107]:
tab1_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   수집시간    15 non-null     datetime64[us]
 1   종목명     15 non-null     object        
 2   현재가     15 non-null     int64         
 3   전일대비    15 non-null     int64         
 4   등락률     15 non-null     float64       
dtypes: datetime64[us](1), float64(1), int64(2), object(1)
memory usage: 732.0+ bytes


## 탭2(상승)

In [60]:
group_type = group_types[1]
stock_lines = group_type.select('tbody tr')

In [61]:
stock_name_list = []
current_price_list = []
compare_yesterday_list = []
updown_ratio_list = []
for one_stock in stock_lines:
    # 종목명
    stock_name = one_stock.select_one('th a').text
    
    # 현재가
    current_price = int(one_stock.select_one('td').text.replace(',', ''))

    # 전일대비
    raw_compare_yesterday = one_stock.select('td')[1].text
    compare_yesterday = int(re.sub('[^0-9]', '', raw_compare_yesterday))

    # 등락률
    raw_updown_ratio = one_stock.select('td')[2].text.strip()
    updown_ratio = float(re.sub('[^0-9.-]', '', raw_updown_ratio))
    
    # 리스트에 추가
    stock_name_list.append(stock_name)
    current_price_list.append(current_price)
    compare_yesterday_list.append(compare_yesterday)
    updown_ratio_list.append(updown_ratio)

In [62]:
len(stock_name_list), len(current_price_list), len(compare_yesterday_list), len(updown_ratio_list)

(15, 15, 15, 15)

In [63]:
tab2_df = pd.DataFrame(
    {
        '수집시간': current_time,
        '종목명': stock_name_list,
        '현재가': current_price_list,
        '전일대비': compare_yesterday_list,
        '등락률': updown_ratio_list
    }
)
tab2_df

,수집시간,종목명,현재가,전일대비,등락률
0,2025-01-03 12:41:04.792503,아이윈플러스,1493,344,29.94
1,2025-01-03 12:41:04.792503,아톤,7990,1840,29.92
2,2025-01-03 12:41:04.792503,한국첨단소재,7210,1660,29.91
3,2025-01-03 12:41:04.792503,아이에이,254,58,29.59
4,2025-01-03 12:41:04.792503,알체라,2275,505,28.53
5,2025-01-03 12:41:04.792503,한화솔루션우,18990,3840,25.35
6,2025-01-03 12:41:04.792503,윈팩,777,152,24.32
7,2025-01-03 12:41:04.792503,다날,3550,660,22.84
8,2025-01-03 12:41:04.792503,에스피시스템스,6970,1250,21.85
9,2025-01-03 12:41:04.792503,큐에스아이,12280,2170,21.46


## 탭3(하락)

In [64]:
group_type = group_types[2]
stock_lines = group_type.select('tbody tr')

In [65]:
stock_name_list = []
current_price_list = []
compare_yesterday_list = []
updown_ratio_list = []
for one_stock in stock_lines:
    # 종목명
    stock_name = one_stock.select_one('th a').text
    
    # 현재가
    current_price = int(one_stock.select_one('td').text.replace(',', ''))

    # 전일대비
    raw_compare_yesterday = one_stock.select('td')[1].text
    compare_yesterday = int(re.sub('[^0-9]', '', raw_compare_yesterday))

    # 등락률
    raw_updown_ratio = one_stock.select('td')[2].text.strip()
    updown_ratio = float(re.sub('[^0-9.-]', '', raw_updown_ratio))
    
    # 리스트에 추가
    stock_name_list.append(stock_name)
    current_price_list.append(current_price)
    compare_yesterday_list.append(compare_yesterday)
    updown_ratio_list.append(updown_ratio)

In [66]:
len(stock_name_list), len(current_price_list), len(compare_yesterday_list), len(updown_ratio_list)

(15, 15, 15, 15)

In [67]:
tab3_df = pd.DataFrame(
    {
        '수집시간': current_time,
        '종목명': stock_name_list,
        '현재가': current_price_list,
        '전일대비': compare_yesterday_list,
        '등락률': updown_ratio_list
    }
)
tab3_df

,수집시간,종목명,현재가,전일대비,등락률
0,2025-01-03 12:41:04.792503,바이온,372,144,-27.91
1,2025-01-03 12:41:04.792503,골든센츄리,4,1,-20.00
2,2025-01-03 12:41:04.792503,흥국화재우,6130,1070,-14.86
3,2025-01-03 12:41:04.792503,유엔젤,4755,805,-14.48
4,2025-01-03 12:41:04.792503,SNT에너지,21600,3150,-12.73
5,2025-01-03 12:41:04.792503,삼현,9390,920,-8.92
6,2025-01-03 12:41:04.792503,CJ씨푸드1우,20100,1900,-8.64
7,2025-01-03 12:41:04.792503,노머스,19020,1630,-7.89
8,2025-01-03 12:41:04.792503,하나 인버스 2X 코스닥150 선물 ETN,7245,595,-7.59
9,2025-01-03 12:41:04.792503,한국경제TV,5750,470,-7.56


## 탭4(시가총액 상위)

In [68]:
group_type = group_types[3]
stock_lines = group_type.select('tbody tr')

In [69]:
stock_name_list = []
current_price_list = []
compare_yesterday_list = []
updown_ratio_list = []
for one_stock in stock_lines:
    # 종목명
    stock_name = one_stock.select_one('th a').text
    
    # 현재가
    current_price = int(one_stock.select_one('td').text.replace(',', ''))

    # 전일대비
    raw_compare_yesterday = one_stock.select('td')[1].text
    compare_yesterday = int(re.sub('[^0-9]', '', raw_compare_yesterday))

    # 등락률
    raw_updown_ratio = one_stock.select('td')[2].text.strip()
    updown_ratio = float(re.sub('[^0-9.-]', '', raw_updown_ratio))
    
    # 리스트에 추가
    stock_name_list.append(stock_name)
    current_price_list.append(current_price)
    compare_yesterday_list.append(compare_yesterday)
    updown_ratio_list.append(updown_ratio)

In [70]:
len(stock_name_list), len(current_price_list), len(compare_yesterday_list), len(updown_ratio_list)

(15, 15, 15, 15)

In [71]:
tab3_df = pd.DataFrame(
    {
        '수집시간': current_time,
        '종목명': stock_name_list,
        '현재가': current_price_list,
        '전일대비': compare_yesterday_list,
        '등락률': updown_ratio_list
    }
)
tab3_df

,수집시간,종목명,현재가,전일대비,등락률
0,2025-01-03 12:41:04.792503,삼성전자,54800,1400,2.62
1,2025-01-03 12:41:04.792503,SK하이닉스,181300,10100,5.90
2,2025-01-03 12:41:04.792503,LG에너지솔루션,359000,13000,3.76
3,2025-01-03 12:41:04.792503,삼성바이오로직스,940000,6000,0.64
4,2025-01-03 12:41:04.792503,현대차,213500,2000,0.95
5,2025-01-03 12:41:04.792503,기아,100400,1200,-1.18
6,2025-01-03 12:41:04.792503,셀트리온,181000,700,0.39
7,2025-01-03 12:41:04.792503,삼성전자우,45200,500,1.12
8,2025-01-03 12:41:04.792503,KB금융,84700,1300,1.56
9,2025-01-03 12:41:04.792503,NAVER,199800,6000,3.10


## 데이터 출력

In [115]:
# TOP 종목
section_sise_top = soup.select_one('div#content div.section_sise_top')
group_types = section_sise_top.select('div.group_type')

In [116]:
# 데이터 출력
current_date = time.strftime('%Y%m%d')
# 현재 경로 확인
code_path = os.getcwd().replace('\\', '/')
# 수집한 파일 저장할 폴더 생성
crawled_folder_path = os.path.join(code_path, 'crawled_data', 'naver_stock', 'home', current_date)
os.makedirs(crawled_folder_path, exist_ok=True)
# 저장할 파일 경로
file_path = os.path.join(crawled_folder_path, f'naver_stock_home_{current_time.strftime("%Y%m%d_%H%M%S")}.xlsx')

# 1번째 : 거래상위, 2번째 : 상승, 3번째 : 하락, 4번째 : 시가총액 상위
sheet_name_list = ['거래상위', '상승', '하락', '시가총액상위']
with pd.ExcelWriter(file_path) as writer:
    for group, sheet_name in zip(group_types, sheet_name_list):
        stock_name_list = []
        current_price_list = []
        compare_yesterday_list = []
        updown_ratio_list = []
        # 종목 정보 추출
        stock_lines = group.select('tbody tr')
        for one_stock in stock_lines:
            # 종목명
            stock_name = one_stock.select_one('th a').text

            # 현재가
            current_price = int(one_stock.select_one('td').text.replace(',', ''))

            # 전일대비
            raw_compare_yesterday = one_stock.select('td')[1].text
            compare_yesterday = int(re.sub('[^0-9]', '', raw_compare_yesterday))

            # 등락률
            raw_updown_ratio = one_stock.select('td')[2].text.strip()
            updown_ratio = float(re.sub('[^0-9.-]', '', raw_updown_ratio))

            # 리스트에 추가
            stock_name_list.append(stock_name)
            current_price_list.append(current_price)
            compare_yesterday_list.append(compare_yesterday)
            updown_ratio_list.append(updown_ratio)
        
        # 데이터프레임 생성
        df = pd.DataFrame(
            {
                '수집시간': current_time,
                '종목명': stock_name_list,
                '현재가': current_price_list,
                '전일대비': compare_yesterday_list,
                '등락률': updown_ratio_list
            }
        )
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print('데이터 저장 경로 :', crawled_folder_path)
print('저장완료')

데이터 저장 경로 : c:/Users/kbjoo/Documents/Google_Drive/GitHub/my_projects/crawling/naver\crawled_data\naver_stock\home\20250103
저장완료


# END